In [27]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import re
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='Parsing.*in DD/MM/YYYY format')
# warnings.filterwarnings('ignore')  # Suppress all warnings

In [28]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None) # Show full column content

In [55]:
df= pd.read_csv("emi_prediction_dataset.csv")

C:\Users\malay chand\AppData\Local\Temp\ipykernel_14004\3266142486.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("emi_prediction_dataset.csv")


In [41]:
df.head(3)

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38.0,Female,Married,Professional,82600.0,Private,0.9,Mid-size,Rented,20000.0,3,2,0.0,0.0,7200.0,19500.0,13200.0,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan EMI,850000.0,15,Not_Eligible,500.0
1,38.0,Female,Married,Graduate,21500.0,Private,7.0,MNC,Family,0.0,2,1,5100.0,0.0,1400.0,5400.0,3500.0,Yes,4100.0,714.0,92500.0,26900.0,E-commerce Shopping EMI,128000.0,19,Not_Eligible,700.0
2,38.0,Male,Married,Professional,86100.0,Private,5.8,Startup,Own,0.0,4,3,0.0,0.0,10200.0,19400.0,6000.0,No,0.0,650.0,672100.0,324200.0,Education EMI,306000.0,16,Eligible,27775.0


### Cleaning age part

In [43]:
df['age'].unique()

array([38.0, 58.0, 48.0, 32.0, 27.0, 47.0, 37.0, 31.0, 59.0, 49.0, 33.0,
       26.0, 39.0, 57.0, 28.0, '58', '38', '48', '32', '27', '37', '48.0',
       '33', '38.0', '49', '27.0', '31', '39', '47', '59', '32.0', '58.0',
       '57', '26', '28', '58.0.0', '39.0', '26.0', '37.0', '38.0.0',
       '32.0.0'], dtype=object)

In [44]:
print(df['age'].dtype)  

object


In [42]:
df.loc[[109660, 238011, 315969]]

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
109660,58.0.0,Male,Married,Graduate,15000.0,Government,4.3,Large Indian,Own,0.0,3,2,3100.0,8800.0,2100.0,4100.0,2100.0,No,0.0,766.0,108400.0,36400.0,Personal Loan EMI,174000.0,32,Not_Eligible,500.0
238011,38.0.0,Female,Married,Post Graduate,84900.0,Private,5.3,Large Indian,Rented,17100.0,3,2,6500.0,24300.0,7900.0,18600.0,13100.0,No,0.0,840.0,594700.0,292600.0,E-commerce Shopping EMI,23000.0,10,Not_Eligible,500.0
315969,32.0.0,Male,Married,Graduate,73700.0,Government,0.5,MNC,Own,0.0,4,3,8200.0,0.0,4100.0,21200.0,4700.0,No,0.0,819.0,403800.0,180700.0,Vehicle EMI,496000.0,62,Eligible,21300.0


In [45]:
# Convert to string, remove everything after first decimal, then to numeric
df['age'] = (df['age'].astype(str).str.split('.').str[0].astype(float))

### Monthly salary

In [56]:
df.loc[[844, 1025, 1244]]

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
844,27.0,Female,Single,Graduate,64300.0.0,Government,0.5,Large Indian,Rented,19200.0,2,1,8600.0,0.0,6400.0,10700.0,7200.0,No,0.0,806.0,344500.0,128200.0,Home Appliances EMI,260000.0,22,Not_Eligible,7320.0
1025,48.0,Female,Married,Post Graduate,23400.0.0.0,Private,4.9,MNC,Family,0.0,4,3,7800.0,7400.0,3400.0,6400.0,1400.0,Yes,5700.0,495.0,12800.0,4900.0,Personal Loan EMI,353000.0,49,Not_Eligible,500.0
1244,38.0,Male,Married,Graduate,22500.0.0,Private,3.8,MNC,Own,0.0,4,3,0.0,0.0,2400.0,5100.0,2400.0,No,0.0,601.0,54400.0,26500.0,Personal Loan EMI,66000.0,56,Eligible,5040.0


In [ ]:
-c

In [47]:
df['monthly_salary'] = pd.to_numeric(df['monthly_salary'], errors='coerce')

In [54]:
df[df["monthly_salary"].isnull()]

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
844,27.0,Female,Single,Graduate,NaN,Government,0.5,Large Indian,Rented,19200.0,2,1,8600.0,0.0,6400.0,10700.0,7200.0,No,0.0,806.0,344500.0,128200.0,Home Appliances EMI,260000.0,22,Not_Eligible,7320.0
1025,48.0,Female,Married,Post Graduate,NaN,Private,4.9,MNC,Family,0.0,4,3,7800.0,7400.0,3400.0,6400.0,1400.0,Yes,5700.0,495.0,12800.0,4900.0,Personal Loan EMI,353000.0,49,Not_Eligible,500.0
1244,38.0,Male,Married,Graduate,NaN,Private,3.8,MNC,Own,0.0,4,3,0.0,0.0,2400.0,5100.0,2400.0,No,0.0,601.0,54400.0,26500.0,Personal Loan EMI,66000.0,56,Eligible,5040.0
1252,32.0,Male,Single,High School,NaN,Self-employed,0.6,Large Indian,Rented,6600.0,1,0,0.0,0.0,2700.0,6000.0,3500.0,No,0.0,671.0,90700.0,47800.0,Vehicle EMI,858000.0,83,Not_Eligible,1710.0
1384,58.0,Male,Married,Professional,NaN,Government,36.0,Large Indian,Family,0.0,3,2,6100.0,0.0,11200.0,31400.0,8900.0,Yes,15200.0,836.0,466100.0,155000.0,Vehicle EMI,1339000.0,77,Not_Eligible,18064.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402339,27.0,Female,Single,Graduate,NaN,Private,1.7,Startup,Rented,14600.0,2,1,2600.0,0.0,3800.0,10800.0,7000.0,No,0.0,662.0,274100.0,72900.0,Vehicle EMI,1363000.0,39,Not_Eligible,14500.0
402819,48.0,Male,Single,Post Graduate,NaN,Private,8.7,Large Indian,Family,0.0,2,1,0.0,7500.0,8600.0,16900.0,11500.0,Yes,18100.0,573.0,392600.0,211900.0,Personal Loan EMI,511000.0,19,Not_Eligible,2380.0
403091,27.0,Female,Single,Graduate,NaN,Private,0.9,MNC,Own,0.0,2,1,5800.0,20100.0,7400.0,10500.0,9200.0,Yes,14600.0,603.0,263000.0,140100.0,E-commerce Shopping EMI,104000.0,11,Not_Eligible,500.0
403182,32.0,Female,Single,Graduate,NaN,Private,0.9,Startup,Rented,8800.0,1,0,0.0,0.0,4000.0,6400.0,4600.0,No,0.0,760.0,152500.0,64000.0,Vehicle EMI,1070000.0,48,Not_Eligible,7056.0


In [49]:
df.tail()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
404795,27.0,Male,Married,Graduate,32400.0,Private,5.0,Large Indian,Rented,10400.0,4,3,2800.0,7000.0,2200.0,4900.0,6100.0,No,0.0,649.0,62000.0,32600.0,Personal Loan EMI,506000.0,47,Not_Eligible,500.0
404796,38.0,Male,Married,Post Graduate,49200.0,Private,1.9,MNC,Own,0.0,5,4,14000.0,0.0,2600.0,13600.0,8600.0,No,0.0,712.0,142200.0,38100.0,Personal Loan EMI,708000.0,33,Not_Eligible,5200.0
404797,32.0,Male,Single,Graduate,25700.0,Private,3.2,MNC,Rented,6300.0,1,0,0.0,0.0,2000.0,4600.0,2500.0,No,0.0,676.0,191600.0,39700.0,Home Appliances EMI,93000.0,21,High_Risk,5665.0
404798,48.0,Male,Married,Graduate,47200.0,Private,3.0,MNC,Own,0.0,4,3,0.0,0.0,7000.0,8900.0,7200.0,No,0.0,784.0,170400.0,45600.0,Home Appliances EMI,144000.0,36,Eligible,14460.0
404799,38.0,FEMALE,Married,Graduate,34900.0,Private,1.2,MNC,Family,0.0,2,1,0.0,0.0,4800.0,8100.0,2100.0,No,0.0,641.0,214400.0,99400.0,Home Appliances EMI,77000.0,35,Eligible,8756.0


In [31]:
numeric_cols_as_object = ['age', 'monthly_salary', 'bank_balance']

for col in numeric_cols_as_object:
    # Remove commas or other non-numeric characters if present (optional but safe)
    df[col] = df[col].astype(str).str.replace(',', '').str.strip()
    # Convert to numeric, coercing errors to NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Optional: Convert 'age' to int if no decimals are meaningful
df['age'] = df['age'].astype('Int64')  # nullable integer

In [ ]:
# df['age'] = pd.to_numeric(df['age'], errors='coerce').astype('Int64')
# df['monthly_salary'] = pd.to_numeric(df['monthly_salary'], errors='coerce')
# df['bank_balance'] = pd.to_numeric(df['bank_balance'], errors='coerce')

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   age                     404800 non-null  float64
 1   gender                  404800 non-null  object 
 2   marital_status          404800 non-null  object 
 3   education               402396 non-null  object 
 4   monthly_salary          402807 non-null  float64
 5   employment_type         404800 non-null  object 
 6   years_of_employment     404800 non-null  float64
 7   company_type            404800 non-null  object 
 8   house_type              404800 non-null  object 
 9   monthly_rent            402374 non-null  float64
 10  family_size             404800 non-null  int64  
 11  dependents              404800 non-null  int64  
 12  school_fees             404800 non-null  float64
 13  college_fees            404800 non-null  float64
 14  travel_expenses     

In [48]:
df.describe()

,age,monthly_salary,years_of_employment,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,current_emi_amount,credit_score,emergency_fund,requested_amount,requested_tenure,max_monthly_emi
count,404800.000000,402807.000000,404800.000000,402374.000000,404800.000000,404800.000000,404800.000000,404800.000000,404800.000000,404800.000000,404800.000000,404800.000000,402380.000000,402449.000000,4.048000e+05,404800.000000,404800.000000
mean,38.875860,59509.340498,5.364079,5828.446490,2.940425,1.940425,4624.575593,4066.253706,5687.497777,12804.999506,7119.309783,4543.407609,700.856223,96769.051731,3.708554e+05,29.126677,6763.602156
std,9.303593,43388.297879,6.079135,8648.604639,1.075199,1.075199,5061.074401,7319.344289,3392.671132,6993.853745,4510.447300,7034.901139,88.435548,81373.053976,3.451945e+05,18.100854,7741.263317
min,26.000000,3967.000000,0.500000,0.000000,1.000000,0.000000,0.000000,0.000000,600.000000,1800.000000,600.000000,0.000000,0.000000,1400.000000,1.000000e+04,3.000000,500.000000
25%,32.000000,35400.000000,1.200000,0.000000,2.000000,1.000000,0.000000,0.000000,3200.000000,7700.000000,3800.000000,0.000000,654.000000,38400.000000,1.240000e+05,15.000000,500.000000
50%,38.000000,51700.000000,3.200000,0.000000,3.000000,2.000000,3000.000000,0.000000,4900.000000,11400.000000,6000.000000,0.000000,701.000000,74000.000000,2.360000e+05,25.000000,4211.200000
75%,48.000000,73000.000000,7.200000,10600.000000,4.000000,3.000000,9000.000000,6500.000000,7400.000000,16400.000000,9300.000000,8000.000000,748.000000,130600.000000,4.940000e+05,40.000000,9792.000000
max,59.000000,499970.000000,36.000000,80000.000000,5.000000,4.000000,15000.000000,25000.000000,30300.000000,71200.000000,42900.000000,56300.000000,1200.000000,891500.000000,1.500000e+06,84.000000,91040.400000


In [39]:
df[df["age"].isnull()]

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
109660,<NA>,Male,Married,Graduate,15000.0,Government,4.3,Large Indian,Own,0.0,3,2,3100.0,8800.0,2100.0,4100.0,2100.0,No,0.0,766.0,108400.0,36400.0,Personal Loan EMI,174000.0,32,Not_Eligible,500.0
238011,<NA>,Female,Married,Post Graduate,84900.0,Private,5.3,Large Indian,Rented,17100.0,3,2,6500.0,24300.0,7900.0,18600.0,13100.0,No,0.0,840.0,594700.0,292600.0,E-commerce Shopping EMI,23000.0,10,Not_Eligible,500.0
315969,<NA>,Male,Married,Graduate,73700.0,Government,0.5,MNC,Own,0.0,4,3,8200.0,0.0,4100.0,21200.0,4700.0,No,0.0,819.0,403800.0,180700.0,Vehicle EMI,496000.0,62,Eligible,21300.0
